In [1]:
# imports etc
import sys

# project dependencies
proj_path = 'd:/code/Projects/property_mapping_sql_generator/'
sys.path.insert(0, proj_path)
from dsl.engine import *

# Simple Example (not nested)

In [2]:
# test simple switch (imagine conv from simple case)
expected_result = switch([(eq(col('a.Prep'), 's1'), 'a1'), 
                          (eq(col('a.Prep'), 's2'), 'a2')], default='an')

print(expected_result)

Switch(expr_val_pairs=[(Equals(expr1=DbColumn(col_name='a.Prep'), expr2='s1'), 'a1'), (Equals(expr1=DbColumn(col_name='a.Prep'), expr2='s2'), 'a2')], default='an')


In [3]:
# converter func that would work with simple unnested case statements 
#   consider changing LogicBranches for nested e.g. to
#   CaseBranch(cols=[Col('Prep', 's1'), Col('Method', 'y')], res='a1-extr'),
#   CaseBranch(cols=[Col('Prep', 's1'), Col('Method', None)], res='a1'), ...

# don't expect to use .val (assuming sub types are `Select`) in real version - case statement could keep simple string inputs for srch / rc?
def conv(case_stmt: Case) -> Switch:
    evps: ExpressionValuePairs = []  # List[Tuple[BoolFunc, ValFuncOrStr]]
    for st, res in case_stmt.search_result_pairs:
        evps.append((Equals(DbColumn(case_stmt.expr.ref_field), st.val), res.val))
    return Switch(evps, default=case_stmt.default.val)

In [5]:
input_case = case('Prep', [('s1', 'a1'), ('s2', 'a2')], default='an')
generated_switch = conv(input_case)
print(generated_switch)
assert(generated_switch == expected_result)

Switch(expr_val_pairs=[(Equals(expr1=DbColumn(col_name='a.Prep'), expr2='s1'), 'a1'), (Equals(expr1=DbColumn(col_name='a.Prep'), expr2='s2'), 'a2')], default='an')


# Nested Example

Not implementing converter for now as I think we *may* need to change the `__init__` types in `Case`, but I will include the case statement and expected_switch

In [8]:
nested_case = Case('Prep', [('s1', Case('Method', [('y', 'a1-extr')], default='a1')),
                            ('s2', Case('Method', [('y', 'a2-extr')], default='a2'))],
                   default=Case('Method', [('y', 'an-extr')], default='an'))

expected_switch = Switch([(eq(col('Prep'), 'x'), switch([(eq(col('Method'), 'y'), 'a1-extr')], default='a1')),
                          (eq(col('Prep'), 'y'), switch([(eq(col('Method'), 'y'), 'a2-extr')], default='a2'))],
                         default=switch([(eq(col('Method'), 'y'), 'an-extr')], default='an'))